<a href="https://colab.research.google.com/github/laisab/AprendizadoDeMaquina/blob/main/Aula02_Regressao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler

Link para o dataset utilizado: https://www.kaggle.com/datasets/navjotkaushal/coffee-sales-dataset/data

In [26]:
file_path = '/content/drive/MyDrive/PPGI_AprendizadoDeMáquina/'
path_name = 'Coffe_sales.csv'

In [27]:
df = pd.read_csv(file_path + path_name)

In [28]:
# Informações sobre o DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3547 entries, 0 to 3546
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   hour_of_day  3547 non-null   int64  
 1   cash_type    3547 non-null   object 
 2   money        3547 non-null   float64
 3   coffee_name  3547 non-null   object 
 4   Time_of_Day  3547 non-null   object 
 5   Weekday      3547 non-null   object 
 6   Month_name   3547 non-null   object 
 7   Weekdaysort  3547 non-null   int64  
 8   Monthsort    3547 non-null   int64  
 9   Date         3547 non-null   object 
 10  Time         3547 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 304.9+ KB


In [29]:
# Primeiros 10 valores
df.head(10)

,hour_of_day,cash_type,money,coffee_name,Time_of_Day,Weekday,Month_name,Weekdaysort,Monthsort,Date,Time
0,10,card,38.7,Latte,Morning,Fri,Mar,5,3,2024-03-01,10:15:50.520000
1,12,card,38.7,Hot Chocolate,Afternoon,Fri,Mar,5,3,2024-03-01,12:19:22.539000
2,12,card,38.7,Hot Chocolate,Afternoon,Fri,Mar,5,3,2024-03-01,12:20:18.089000
3,13,card,28.9,Americano,Afternoon,Fri,Mar,5,3,2024-03-01,13:46:33.006000
4,13,card,38.7,Latte,Afternoon,Fri,Mar,5,3,2024-03-01,13:48:14.626000
5,15,card,33.8,Americano with Milk,Afternoon,Fri,Mar,5,3,2024-03-01,15:39:47.726000
6,16,card,38.7,Hot Chocolate,Afternoon,Fri,Mar,5,3,2024-03-01,16:19:02.756000
7,18,card,33.8,Americano with Milk,Night,Fri,Mar,5,3,2024-03-01,18:39:03.580000
8,19,card,38.7,Cocoa,Night,Fri,Mar,5,3,2024-03-01,19:22:01.762000
9,19,card,33.8,Americano with Milk,Night,Fri,Mar,5,3,2024-03-01,19:23:15.887000


In [30]:
# Colunas = Atributos
df.columns

Index(['hour_of_day', 'cash_type', 'money', 'coffee_name', 'Time_of_Day',
       'Weekday', 'Month_name', 'Weekdaysort', 'Monthsort', 'Date', 'Time'],
      dtype='object')

In [31]:
# Quantidade de linhas e colunas
linhas, colunas = df.shape
print(f'Linhas: {linhas}\nColunas: {colunas}')

Linhas: 3547
Colunas: 11


In [32]:
# Descrição estatística do DataFrame (valores numéricos)
df.describe()

,hour_of_day,money,Weekdaysort,Monthsort
count,3547.000000,3547.000000,3547.000000,3547.000000
mean,14.185791,31.645216,3.845785,6.453905
std,4.234010,4.877754,1.971501,3.500754
min,6.000000,18.120000,1.000000,1.000000
25%,10.000000,27.920000,2.000000,3.000000
50%,14.000000,32.820000,4.000000,7.000000
75%,18.000000,35.760000,6.000000,10.000000
max,22.000000,38.700000,7.000000,12.000000


In [33]:
# Valores ausentes
print(f'Valores ausentes:\n{df.isna().sum()}')

Valores ausentes:
hour_of_day    0
cash_type      0
money          0
coffee_name    0
Time_of_Day    0
Weekday        0
Month_name     0
Weekdaysort    0
Monthsort      0
Date           0
Time           0
dtype: int64


In [34]:
# Mudança no nome dos atributos
nome_atributos = {
    'hour_of_day': 'hora',
    'cash_type': 'forma_pagamento',
    'money': 'valor',
    'coffee_name': 'tipo_cafe',
    'Time_of_Day': 'periodo',
    'Weekday': 'dia_semana',
    'Month_name': 'nome_mes',
    'Weekdaysort': 'num_dia',
    'Monthsort': 'num_mes',
    'Date': 'data',
    'Time': 'hora_pagamento'
}

df.rename(columns=nome_atributos, inplace=True)
df

,hora,forma_pagamento,valor,tipo_cafe,periodo,dia_semana,nome_mes,num_dia,num_mes,data,hora_pagamento
0,10,card,38.70,Latte,Morning,Fri,Mar,5,3,2024-03-01,10:15:50.520000
1,12,card,38.70,Hot Chocolate,Afternoon,Fri,Mar,5,3,2024-03-01,12:19:22.539000
2,12,card,38.70,Hot Chocolate,Afternoon,Fri,Mar,5,3,2024-03-01,12:20:18.089000
3,13,card,28.90,Americano,Afternoon,Fri,Mar,5,3,2024-03-01,13:46:33.006000
4,13,card,38.70,Latte,Afternoon,Fri,Mar,5,3,2024-03-01,13:48:14.626000
...,...,...,...,...,...,...,...,...,...,...,...
3542,10,card,35.76,Cappuccino,Morning,Sun,Mar,7,3,2025-03-23,10:34:54.894000
3543,14,card,35.76,Cocoa,Afternoon,Sun,Mar,7,3,2025-03-23,14:43:37.362000
3544,14,card,35.76,Cocoa,Afternoon,Sun,Mar,7,3,2025-03-23,14:44:16.864000
3545,15,card,25.96,Americano,Afternoon,Sun,Mar,7,3,2025-03-23,15:47:28.723000


In [35]:
# Dados redundantes
novo_df = df.drop(columns=['hora', 'num_dia', 'num_mes'], axis=1)

In [36]:
novo_df.head(10)

,forma_pagamento,valor,tipo_cafe,periodo,dia_semana,nome_mes,data,hora_pagamento
0,card,38.7,Latte,Morning,Fri,Mar,2024-03-01,10:15:50.520000
1,card,38.7,Hot Chocolate,Afternoon,Fri,Mar,2024-03-01,12:19:22.539000
2,card,38.7,Hot Chocolate,Afternoon,Fri,Mar,2024-03-01,12:20:18.089000
3,card,28.9,Americano,Afternoon,Fri,Mar,2024-03-01,13:46:33.006000
4,card,38.7,Latte,Afternoon,Fri,Mar,2024-03-01,13:48:14.626000
5,card,33.8,Americano with Milk,Afternoon,Fri,Mar,2024-03-01,15:39:47.726000
6,card,38.7,Hot Chocolate,Afternoon,Fri,Mar,2024-03-01,16:19:02.756000
7,card,33.8,Americano with Milk,Night,Fri,Mar,2024-03-01,18:39:03.580000
8,card,38.7,Cocoa,Night,Fri,Mar,2024-03-01,19:22:01.762000
9,card,33.8,Americano with Milk,Night,Fri,Mar,2024-03-01,19:23:15.887000


In [37]:
# Atributos categóricos
novo_df.describe(include='object')

,forma_pagamento,tipo_cafe,periodo,dia_semana,nome_mes,data,hora_pagamento
count,3547,3547,3547,3547,3547,3547,3547
unique,1,8,3,7,12,381,3547
top,card,Americano with Milk,Afternoon,Tue,Mar,2024-10-11,18:11:38.635000
freq,3547,809,1205,572,494,26,1


In [38]:
# Transformação de atributos categóricos em numéricos
atr_cat = ['forma_pagamento', 'tipo_cafe', 'periodo', 'dia_semana', 'nome_mes']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
valor_encoded = encoder.fit_transform(novo_df[atr_cat])
df_encoded = pd.DataFrame(valor_encoded, columns=encoder.get_feature_names_out(atr_cat))
df_encoded.head(10)

,forma_pagamento_card,tipo_cafe_Americano,tipo_cafe_Americano with Milk,tipo_cafe_Cappuccino,tipo_cafe_Cocoa,tipo_cafe_Cortado,tipo_cafe_Espresso,tipo_cafe_Hot Chocolate,tipo_cafe_Latte,periodo_Afternoon,...,nome_mes_Dec,nome_mes_Feb,nome_mes_Jan,nome_mes_Jul,nome_mes_Jun,nome_mes_Mar,nome_mes_May,nome_mes_Nov,nome_mes_Oct,nome_mes_Sep
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [39]:
# Padronização da escala dos dados numéricos
atr_num = ['valor']
scaler = StandardScaler()
valor_scaled = scaler.fit_transform(novo_df[atr_num])
df_scaled = pd.DataFrame(valor_scaled, columns=atr_num)
df_scaled.head(10)

,valor
0,1.446522
1,1.446522
2,1.446522
3,-0.562883
4,1.446522
5,0.441820
6,1.446522
7,0.441820
8,1.446522
9,0.441820


In [40]:
df_final = pd.concat([df_encoded, df_scaled], axis=1)
df_final.head(10)

,forma_pagamento_card,tipo_cafe_Americano,tipo_cafe_Americano with Milk,tipo_cafe_Cappuccino,tipo_cafe_Cocoa,tipo_cafe_Cortado,tipo_cafe_Espresso,tipo_cafe_Hot Chocolate,tipo_cafe_Latte,periodo_Afternoon,...,nome_mes_Feb,nome_mes_Jan,nome_mes_Jul,nome_mes_Jun,nome_mes_Mar,nome_mes_May,nome_mes_Nov,nome_mes_Oct,nome_mes_Sep,valor
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.446522
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.446522
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.446522
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.562883
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.446522
5,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.441820
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.446522
7,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.441820
8,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.446522
9,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.441820


In [41]:
# Quantidade de linhas e colunas
linhas, colunas = df_final.shape
print(f'Linhas: {linhas}\nColunas: {colunas}')

Linhas: 3547
Colunas: 32


In [42]:
X = df_final.drop('valor', axis=1) # Features
y = df_final['valor'] # Target

In [43]:
# Separação em dados de treino e de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f'Dados de treino: {X_train.shape}')
print(f'Dados de teste: {X_test.shape}')

Dados de treino: (2482, 31)
Dados de teste: (1065, 31)


In [44]:
# Regressão linear
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [45]:
# Avaliação do modelo
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(' RESULTADOS DA AVALIAÇÃO')
print(f'MAE: {mae:.3f}')
print(f'MSE: {mse:.3f}')
print(f'RMSE: {rmse:.3f}')
print(f'R²: {r2:.3f}')

 RESULTADOS DA AVALIAÇÃO
MAE: 0.076
MSE: 0.022
RMSE: 0.149
R²: 0.977
